In [53]:
import http.client
import json
import pandas as pd
from pandas.io.json import json_normalize
import io

In [72]:
league_codes = {
    "Champions_League": 2001,
    "Primera_Division": 2014,
    "Bundesliga": 2002,
    "Premier_League":2021,
    "Serie_A": 2019,
    "Ligue_1":2015
}

Create a loop in order to get the data from the relevant leagues using the dictionary league_codes and save the raw json in a txt file separately for the different leagues. Then I used the output and feeded a 3rd party software that translates json to csv. It can also be done in python, but it is nested into many levels and it proved hard to normalize and flatten it.

In [58]:
for name,code in league_codes.items():
    connection = http.client.HTTPConnection('api.football-data.org')
    headers = { 'X-Auth-Token': 'b609904fbfc44877b6359d54d7f19ddf' }
    connection.request('GET', 'http://api.football-data.org/v2/competitions/%d/matches?season=2017'%(code), None, headers )
    response2 = json.dumps(connection.getresponse().read().decode())
    dff = json.loads(response2)
    with io.open('C:\\Users\\makar\\.spyder-py3\\Football\\%s.txt'%(name), "a", encoding="utf-8") as f:
        f.write(dff)


Here I've included some of the tries with the json to flatten it:

In [71]:
data = pd.DataFrame.from_dict(dff2, orient='index')
data

,0
count,380
filters,{}
competition,"{'id': 2015, 'area': {'id': 2081, 'name': 'Fra..."
matches,"[{'id': 203463, 'season': {'id': 17, 'startDat..."


In [60]:
dff2=json.loads(dff)
#As we see, the dff object is string, the dff2 object is translated to dictionary object, so python can work with it.
print(type(dff))
print(type(dff2))

<class 'str'>
<class 'dict'>


In [62]:
#Here I use the json_normalize with the record_path=... in order to access the nested dictionary, but we can see,
#that there are still some elements, that don't work out well, so I decided to quit and use the converter.

df = json_normalize(dff2,record_path=['matches'])
print(df.head(1))

                             awayTeam           group  \
0  {'id': 511, 'name': 'Toulouse FC'}  Regular Season   

                              homeTeam      id           lastUpdated  \
0  {'id': 548, 'name': 'AS Monaco FC'}  203463  2018-06-22T10:02:34Z   

   matchday                                           referees  \
0         1  [{'id': 9374, 'name': 'Clément Turpin', 'natio...   

                                               score  \
0  {'winner': 'HOME_TEAM', 'duration': 'REGULAR',...   

                                              season           stage  \
0  {'id': 17, 'startDate': '2017-08-04', 'endDate...  REGULAR_SEASON   

     status               utcDate  
0  FINISHED  2017-08-04T18:45:00Z  


In [66]:
#I can create a df for referees and for away teams and etc, but it is too time consuming and therefore I went for the 
#option with the converter:

df = json_normalize(dff2,record_path=['matches','referees'])
df.head()

,id,name,nationality
0,9374,Clément Turpin,None
1,57029,Huseyin Ocak,None
2,57030,Gilles Lang,None
3,57031,Alexandre Perreau Niel,None
4,57033,Mikael Lesage,None
